In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
        temperature = 0,
        groq_api_key = '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',
        model_name = "llama-3.1-70b-versatile")

response = llm.invoke("The person to land on moon was")
print(response.content)

Neil Armstrong was the first person to land on the moon. He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission.


In [3]:
# webbased loader class from llm accepts the url of job posting and extracts data from it.

In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38583")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Software Engineer II

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer IIBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more th

In [8]:
from langchain_core.prompts import PromptTemplate
# prompt_extract because we are using it for extraction purpose
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

# below is the langchain chain - getting prompt and passing it to llm
chain_extract = prompt_extract | llm
# page_data obj defined above
res = chain_extract.invoke(input={'page_data' : page_data})
print(res.content)

```json
{
  "role": "Software Engineer II",
  "experience": "5+ years",
  "skills": [
    "Python",
    "Java",
    "AWS",
    "Apache NiFi",
    "Microservice architecture",
    "ERP",
    "SaaS",
    "SAP",
    "Coupa",
    "PEGASystems",
    "Jira",
    "GIT",
    "Jenkins",
    "CI/CD pipelines",
    "Agile",
    "Scrum",
    "DevOps"
  ],
  "description": "We are looking for a Software Engineer to contribute to the design, build, test, deploy, and maintenance of technology solutions. The candidate needs to have experience and an understanding of contemporary development, testing, and observability techniques and tools in a DevOps environment. A learning mindset, excellent analytical skills, and strong written and verbal communication are essential."
}
```


In [9]:
type(res.content)

str

In [11]:
# convert str to json object
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


{'role': 'Software Engineer II',
 'experience': '5+ years',
 'skills': ['Python',
  'Java',
  'AWS',
  'Apache NiFi',
  'Microservice architecture',
  'ERP',
  'SaaS',
  'SAP',
  'Coupa',
  'PEGASystems',
  'Jira',
  'GIT',
  'Jenkins',
  'CI/CD pipelines',
  'Agile',
  'Scrum',
  'DevOps'],
 'description': 'We are looking for a Software Engineer to contribute to the design, build, test, deploy, and maintenance of technology solutions. The candidate needs to have experience and an understanding of contemporary development, testing, and observability techniques and tools in a DevOps environment. A learning mindset, excellent analytical skills, and strong written and verbal communication are essential.'}

In [12]:
type(json_res)

dict

In [14]:
# company portfolio is inserted into chromaDB
# the skills from the job posting are mapped into one or multiple of any of the techstack
# and should retrieve the portfolio Urls which will be used in the email.

In [17]:
import pandas as pd

df = pd.read_csv("company_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [20]:
# iterate through above one by one and insert into chromadb
import uuid
import chromadb

# client - create chromadb in memory
# persistent client - chromadb in disk
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())] )

In [23]:
links = collection.query(query_texts=["Experience in Python","Experience in React"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [25]:
job = json_res
job['skills']

['Python',
 'Java',
 'AWS',
 'Apache NiFi',
 'Microservice architecture',
 'ERP',
 'SaaS',
 'SAP',
 'Coupa',
 'PEGASystems',
 'Jira',
 'GIT',
 'Jenkins',
 'CI/CD pipelines',
 'Agile',
 'Scrum',
 'DevOps']

In [26]:
links = collection.query(query_texts=job["skills"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links':

In [27]:
job

{'role': 'Software Engineer II',
 'experience': '5+ years',
 'skills': ['Python',
  'Java',
  'AWS',
  'Apache NiFi',
  'Microservice architecture',
  'ERP',
  'SaaS',
  'SAP',
  'Coupa',
  'PEGASystems',
  'Jira',
  'GIT',
  'Jenkins',
  'CI/CD pipelines',
  'Agile',
  'Scrum',
  'DevOps'],
 'description': 'We are looking for a Software Engineer to contribute to the design, build, test, deploy, and maintenance of technology solutions. The candidate needs to have experience and an understanding of contemporary development, testing, and observability techniques and tools in a DevOps environment. A learning mindset, excellent analytical skills, and strong written and verbal communication are essential.'}

In [28]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineer II for Your Technology Solutions

Dear Hiring Manager,

I came across your job posting for a Software Engineer II and was impressed by the role's requirements. At AtliQ, we have a proven track record of delivering high-quality technology solutions that meet and exceed our clients' expectations. Our team of expert software engineers has extensive experience in designing, building, testing, deploying, and maintaining technology solutions that align with your requirements.

Our expertise in Python, Java, AWS, Apache NiFi, Microservice architecture, ERP, SaaS, SAP, Coupa, PEGASystems, Jira, GIT, Jenkins, CI/CD pipelines, Agile, Scrum, and DevOps makes us an ideal partner for your technology needs. We have a deep understanding of contemporary development, testing, and observability techniques and tools in a DevOps environment.

We would like to showcase some of our relevant work:

* Our expertise in Python can be seen in our [Python Portfolio](https://examp